# MIGHTEE-HI Data: HI Profile Extraction Analysis
***

By <b>Wanga Mulaudzi</b> (based on work by Sambatra Rajohnson, Dr Marcin Glowacki and Dr Bradley Frank)

<b>Date:</b> 1 August 2021
<br>
<b>Affiliation:</b> Department of Astronomy, University of Cape Town, Private Bag X3, Rondebosch 7701, South Africa
<br>
<b>Contact:</b> MLDWAN001@myuct.ac.za
<br>
<b>Ilifu Jupyter Kernel:</b> ASTRO-PY3 

This notebook:
<br>
* Applies a primary beam correction to the data cube using python, 
* Extracts spectra from cubelets in the data cube after having performed source finding to get their locations.

In [1]:
# Some necessary import statements
import aplpy
from astropy.io import ascii
from astropy.io import fits
from astropy import cosmology
from astropy.cosmology import WMAP7
from astropy.cosmology import LambdaCDM
from astropy import units as u
from astropy import constants as const
from astropy.coordinates import SkyCoord
import heapq
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import math
import os
from scipy.optimize import curve_fit
import scipy as sp
import sklearn
import seaborn as sns
import shutil
import spectral_cube
from spectral_cube import SpectralCube
from spectral_cube.cube_utils import Beam
%matplotlib inline

In [2]:
# One can initiate parameters for plotting
# This is up to the user
pl.rc('axes',titlesize='large')
pl.rc('text', usetex=False)
pl.rc('font', **{'family':'serif','size':20})
pl.rc('axes', labelsize=16)
pl.rc('xtick',labelsize=16)
pl.rc('ytick',labelsize=16)

# 1. Defining directories

In [ ]:
'''For this example, we will use the COSMOS 123 data cubes'''
# Directory to where the data cubes are
data_dir = '/idia/projects/mightee/mightee-hi/COSMOS/123/'

# In this case, the COSMOS data cubes are in a public directiory
# Define your own personal directory that you can store your output cubes
my_data_dir = ''

# Directory to the detections list
det_dir = ''

# Directory to store results of this notebooks analysis
analysis_res = ''

# 2. Reading in the data

In [3]:
# Read in the main cube
cube_file = data_dir+'COSMOS123.CORR.1330.ms.contsub.dirty.w128.image.u.piwimed.fits'
cube = SpectralCube.read(cube_file)

# Store the header in a variable
hdulist = fits.open(cube_file)

# Get the table of beams
beam_table = hdulist[1].data

# Display information about the cube
cube

VaryingResolutionSpectralCube with shape=(287, 4096, 4096) and unit=Jy / beam:
 n_x:   4096  type_x: RA---SIN  unit_x: deg    range:   148.979931 deg:  151.258959 deg
 n_y:   4096  type_y: DEC--SIN  unit_y: deg    range:     1.067545 deg:    3.342695 deg
 n_s:    287  type_s: FREQ      unit_s: Hz     range: 1330320493.996 Hz:1390096088.856 Hz

From the information above, we can see the RA and Dec ranges, and that the total bandwidth of the cube is 50 MHz. 

We now have to read in primary beam.

In [4]:
# Read in the primary beam file
pb_file = data_dir+'COSMOS123.CORR.1330.ms.contsub.dirty.w128.pb.u.e.fits'
pb = SpectralCube.read(pb_file)

# Display information about the cube
pb

SpectralCube with shape=(287, 4096, 4096):
 n_x:   4096  type_x: RA---SIN  unit_x: deg    range:   148.979931 deg:  151.258959 deg
 n_y:   4096  type_y: DEC--SIN  unit_y: deg    range:     1.067545 deg:    3.342695 deg
 n_s:    287  type_s: FREQ      unit_s: Hz     range: 1330320493.996 Hz:1390096088.856 Hz

# 3. Primary beam correction
* If you choose to perform primary beam correction in ```Python```, then follow step 3.1. (note that one can also do primary beam correction in ```CASA```).
* If you have already done primary beam correction, then skip to step 3.2.

## 3.1. If the primary beam correction has not yet been done:

In [ ]:
# Extract the header and data of the cube
cubehead = fits.getheader(cube_file)
cubedata = fits.getdata(cube_file)

# Check the shape of the cube
shape = cubedata.shape

# Check how many dimenstions shape has
if len(shape) == 4:
        # Assign each value and the dimension
        # nc is the number of cubes
        # nf is the number of frequency/velocity channels
        # ny is the number of DEC points
        # nx is the number of RA points
        nc, nf, ny, nx = shape 
        dim = 4
elif len(shape) == 2:
        ny, nx = S
        dim = 2
else:
    raise(Exception, "I don't know how to handle a cube with this shape: "+str(shape))
    
# Extract the header and data of the primary beam
pbhead = fits.getheader(pb_file)
pbdata = fits.getdata(pb_file)

# Causes WCS init problem if zero
if pbhead['CDELT4'] == 0.0:
    pbhead['CDELT4'] = -8.236827542606E+07
    
# Primary beam correction by dividing the image by the pb
pbcordat = cubedata / pbdata

# Check the shape of the primary beam corrected cube
pbshape = pbcordat.shape

# Check how many dimenstions pbshape has
if len(pbshape) == 4:
        pbnc, pbnf, pbny, pbnx = pbshape
        pbdim = 4
elif len(pbshape) == 2:
        pbny, pbnx = pbS
        ndim = 2
else:
        raise(Exception, "I don't know how to handle a primary beam corrected cube with this shape: "+str(pbshape))

# Save the primary beam corrected file
outfile = my_data_dir+'COSMOS123.CORR.1330.ms.contsub.dirty.w128.image.u.e.piwimed.pbcorr.fits'
fits.writeto(outfile, pbcordat, header = hdulist[0].header)
fits.append(outfile, beam_table, header = hdulist[1].header)

## 3.2. If the primary beam correction has already been done:

We can now read in the primary beam corrected cube, which was done in ```Python```:

In [5]:
# Read in the primary beam corrected cube
pbcordat_file = my_data_dir+'COSMOS123.CORR.1330.ms.contsub.dirty.w128.image.u.e.piwimed.pbcorr.fits'
pbcordat = SpectralCube.read(pbcordat_file)

# Display information about the cube
pbcordat

VaryingResolutionSpectralCube with shape=(287, 4096, 4096) and unit=Jy / beam:
 n_x:   4096  type_x: RA---SIN  unit_x: deg    range:   148.979931 deg:  151.258959 deg
 n_y:   4096  type_y: DEC--SIN  unit_y: deg    range:     1.067545 deg:    3.342695 deg
 n_s:    287  type_s: FREQ      unit_s: Hz     range: 1330320493.996 Hz:1390096088.856 Hz

# 4. Extracting subcubes

Now that we have a primary beam corrected image, we can extract subcubes around each detection using their positions and frequencies at which the detections occur and plot the HI spectra.

My source lists have the following columns:
<br>
* Detection (number of the detection in the list)
* RA (degrees)
* Dec (degrees)
* Ref (name of detection if found in another catalogue, e.g. AGC208525. If there is no reference in another catalogue, then I just use '-')
* MinFreq (minimum baseline frequency of the profile in GHz)
* MidFreq (centre frequency in GHz)
* MaxFreq (maximum baseline frequency of the profile in GHz)
* MinSpec (minimum frequency frequency of the profile in GHz)
* MaxSpec (maximum frequency frequency of the profile in GHz)
* Profile (morphology: Flat, Double, Gaussian, etc.)
* log(MHI) (from MIGHTEE-HI catalogue)
* MHI(1e9) (10^log(MHI))
* Name (MIGHTEE-HI name from MIGHTEE-HI catalogue)
* z (redshift at MidFreq)
* log(Mstellar) (from MIGHTEE-HI catalogue)
* log(SFR) (from MIGHTEE-HI catalogue)
* log(Age) (from MIGHTEE-HI catalogue)
* EBV (from MIGHTEE-HI catalogue)
* umag (from MIGHTEE-HI catalogue)
* gmag (from MIGHTEE-HI catalogue)
* rmag (from MIGHTEE-HI catalogue)
* Semimaj (from MIGHTEE-HI catalogue)
* Semimin (from MIGHTEE-HI catalogue)
* PA (from MIGHTEE-HI catalogue)

In [6]:
detections = ascii.read(det_dir+'COSMOS123_1330_catalogue_notes.txt', header_start = 0, data_start = 1)

Detection,RA,Dec,Ref,MinFreq,MidFreq,MaxFreq,MinSpec,MaxSpec,Profile,log(MHI),MHI(1e9),Name,z,log(Mstellar),log(SFR),log(Age),EBV,umag,gmag,rmag,Semimaj,Semimin,PA
int64,float64,float64,str9,float64,float64,float64,float64,float64,str17,float64,float64,str21,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,149.846692,2.693685,-,1.354,1.355819,1.357,1.355192,1.356446,Flat,8.983,0.961612,MGTH_J095923.2+024137,0.047637,9.68096,-0.148713,9.90309,0.05,18.364504,17.424356,16.977141,12.135,6.913,80.0
4,150.430155,2.685806,AGC208525,1.355,1.356655,1.368,1.356237,1.357282,Flat,9.712,5.152286,MGTH_J100143.2+024109,0.046991,9.23994,-0.081352,9.477121,0.1,18.593977,17.721064,17.366805,16.782,12.575,0.0
5,150.038618,2.713136,AGC204456,1.374,1.375466,1.377,1.375257,1.376093,Double,9.472,2.964831,MGTH_J100009.3+024247,0.032672,9.45761,-0.521555,9.875061,0.0,0.0,17.083673,16.655813,19.904,10.339,330.0
7,149.756951,2.549942,-,1.379,1.380482,1.382,1.380064,1.3809,"Flat,lopsided",8.801,0.632412,MGTH_J095901.7+023300,0.02892,7.56165,-0.414986,8.006543,0.2,19.502996,18.873676,18.688301,7.635,5.561,90.0
8,150.807844,2.341862,-,1.365,1.367106,1.369,1.3669,1.367732,"Gaussian,lopsided",9.32,2.089296,MGTH_J100313.9+022031,0.038987,-99.0,-99.0,0.0,-99.0,0.0,0.0,0.0,4.0,4.0,0.0
10,150.171143,2.412895,-,1.355,1.356655,1.358,1.356237,1.357073,Gaussian,9.068,1.169499,MGTH_J100041.1+022446,0.046991,8.19382,-0.424585,8.756548,0.3,20.86746,19.977257,19.594132,5.342,5.111,90.0
11,150.404485,2.10662,-,1.358,1.35979,1.361,1.359372,1.359999,Gaussian,8.491,0.309742,MGTH_J100137.1+020624,0.044577,8.92938,-0.673457,9.812913,0.1,19.620231,18.6917,18.272216,8.493,8.422,90.0
12,150.230087,2.395457,AGC204457,1.358,1.360208,1.362,1.359372,1.360626,"Flat,lopsided",9.274,1.879317,MGTH_J100055.2+022344,0.044256,10.416,-0.122561,9.845098,0.05,17.547062,16.214165,15.586224,21.68,13.983,0.0
13,150.313451,2.306401,-,1.379,1.381109,1.383,1.380273,1.381318,"Flat,lopsided",9.248,1.770109,MGTH_J100115.2+021823,0.028453,8.61082,-0.783372,9.414973,0.0,18.468288,17.761891,17.487835,15.423,5.408,345.0


In the next cell, a function called `get_subcube_and_noisecube` has been defined. It extracts a subcube from an input cube, as well as the corresponding noise cube. The noise cubes are important for taking the standard deviation of noise in the noise cubes and using the mean as the overall noise.

In [7]:
# This function extracts the signal and noise cubes
def get_subcube_and_noisecube(cube, common_beam, ra, dec, freq, width, velocity_convention='relativistic', noise_offset = 0.5*u.arcmin, dfreq=0.0005):
    '''
    cube is the cube used for the analysis
    ra is the ra of the detection in decimal degrees or hours, minutes and seconds
    dec is the dec of the detection in decimal degrees or degrees, minutes and seconds
    freq is the center frequency in GHz
    width is the size of the signal cube in arcminutes
    velocity_convention by default is the relativistic convention
    noise_offset is the amount added to the ra of the detection to get the ra of the noise cube
    dfreq is the offset from the centre frequency in GHz
    
    Outputs are:
    the signal cube with spectral axis frequency
    the signal cube with spectral axis velocity
    the frequency axis
    the velocity axis
    the flux density
    the noise cube with spectral axis velocity
    the detections coordinates
    '''
    # Lower and upper frequency limits
    freq_lower = '%.5fGHz' % (freq-dfreq) 
    freq_upper = '%.5fGHz' % (freq+dfreq)
    
    # Subcube extraction string
    crtf_str = 'centerbox[['+ra+','+dec+'], ['+width+','+width+']], coord=fk5, range=['+freq_lower+', '+freq_upper+']]'
    
    # Convert the RA and DEC into decimal degrees if not yet in decimal degrees
    coord = SkyCoord(ra, dec, unit='deg', frame='fk5')
    
    # RA and Dec for the noise cube. Need to shift the RA by the specified width
    ra_deg = coord.ra.deg + noise_offset.to(u.deg).value
    dec_deg = coord.dec.deg # Same declination as signal cube

    # Noise cube extraction
    noise_crtf = 'centerbox[['+str(ra_deg)+','+str(dec_deg)+'], ['+width+','+width+']], coord=fk5, range=['+freq_lower+', '+freq_upper+']]'

    # Generating the new signal cube from the crtf_str region, spectral axis = frequency
    subcube = cube.subcube_from_crtfregion(crtf_str) 
    
    # Convolve the subcube to the common beam
    target_subcube = subcube.convolve_to(common_beam)
    
    # Generating the corresponding noise cube from the noise_crtf region, spectral axis = frequency
    noise_sub = cube.subcube_from_crtfregion(noise_crtf) 
    noise_subcube = noise_sub.convolve_to(common_beam)
    
    # Frequency axis in Hz
    freqs = target_subcube.spectral_axis 
    
    # Convert the signal and noise cubes spectral axes into velocity km/s in radio convention
    vel_subcube = target_subcube.with_spectral_unit(u.km / u.s, velocity_convention=velocity_convention, rest_value=1.42040575e9*u.Hz)
    noise_velsubcube = noise_subcube.with_spectral_unit(u.km / u.s, velocity_convention=velocity_convention, rest_value=1.42040575e9*u.Hz)
    
    # Velocity axis in km/s
    vel = vel_subcube.spectral_axis
    
    # Flux density values in Jy/beam
    target_spectrum_sum = target_subcube.sum(axis=(1,2))/target_subcube.unit
    
    return coord, target_subcube, vel_subcube, freqs, vel, target_spectrum_sum, noise_velsubcube, noise_subcube

Subcubes of 2 arcmin radii are typically extracted, but sometimes you have feint sources or sources that are very close to each other that need smaller radii, or really large sources that need bigger radii. The parameters used for the extractions are the same as those used in `Step1_Moment_Maps_COSMOS_1330.ipynb` (apart from an increased frequency width) for consistency. 

In [8]:
# Common beam of the main cube
common_beam = cube.beams.common_beam(tolerance=1e-5)

# Extraction parameters
subcube_width = 2 # Size of the subcube in arcminutes
subcube_width_in_text = '2arcmin' # String format of the size of the subcube

# We may encounter detections that need a bigger or smaller subcube extraction, so you can select their index numbers:
list_index = []
# The new width to use instead of '2arcmin'
width_new = []
width_new_in_text = []

# Frequency width for the extraction
half_freq_width = 2e-3 # This is dfreq in the det_subcube_and_noisecube() function

In [9]:
%%capture
# Capture all warnings

# Subcube and noise cubes extractions
subcube = [] # Subcubes with frequency axis
Sum = [] # Sum value of the flux
vel = [] # Velocity axis
freqs = [] # Frequency axis
vel_subcube = [] # Subcubes with velocity axis
noise_velsubcube = [] # Noise cubes with velocity axis
noise_subcube = [] # Noise cubes with frequency axis
dets = [] # Detection numbers
coordinates = [] # Coordinates of each detection
center_freqs = [] # Center frequencies

for i in range(len(detections)):
    ra = detections[i]['RA'] 
    dec = detections[i]['Dec'] 
    freq = detections[i]['MidFreq'] 
    det = detections[i]['Detection']
    
    # Check if the detection needs a bigger radius of extraction (refer to list_index in very first cell)
    if detections[i]['Detection'] in list_index:
        # Get the detections index in the index list
        i = list_index.index(detections[i]['Detection'])
        coord, a, b, c, d, e, f, g = get_subcube_and_noisecube(pbcordat, common_beam, str(ra), str(dec), freq, noise_offset=width_new[i]*u.arcmin, width=width_new_in_text[i], dfreq=half_freq_width)
    # Else extract a subcube of size 2 arcminutes
    else:
        coord, a, b, c, d, e, f, g = get_subcube_and_noisecube(pbcordat, common_beam, str(ra), str(dec), freq, noise_offset=subcube_width*u.arcmin, width=subcube_width_in_text, dfreq=half_freq_width)
    
    # Store everything in the lists
    coordinates.append(coord)
    subcube.append(a)
    vel_subcube.append(b)
    freqs.append(c)
    vel.append(d)
    Sum.append(e)
    noise_velsubcube.append(f)
    noise_subcube.append(g)
    dets.append(det)
    center_freqs.append(freq*u.GHz)

# 5. Smoothing the cubes

As with the `Step1_Moment_Maps_COSMOS_1330.ipynb` notebook, for consistency we need to convolve the subcubes and noise cubes into a common restoring beam: the bigger the restored beam is, the more sensitive to faint emission the cube will be. For that, we will convolve the signal cubes into a circular beam of 20" x 20" with a bpa = 0 degrees. 

In [10]:
# This function convolves the cube to a circular beam
def beam_convolution_and_mean_rms(cube, noise_cube, beam, detections_list):
    '''
    cube is the signal cube to be convolved
    noise_cube is the noise_cube to be convolved
    beam is the beam we will convolve the cubes to 
    detections_list is the ASCII table containing the list of detections
    
    Outputs are:
    a list of the convolved signal cubes
    a list of the convolved noise cubes
    the mean noise from the noise cube
    '''
    convolved_cube = [] # List to store convolved signal cubes
    convolved_noise_cube = [] # List to store convolved noise cubes
    mean_rms = [] # List to store mean noise values from each noise cube
    
    # Loop through each detection
    for i in range(len(detections_list)):
        # Convolve the signal and noise cubes
        convolved_cube.append(cube[i].convolve_to(beam))
        convolved_noise_cube.append(noise_cube[i].convolve_to(beam))
        
        # Calculate the mean of standard deviation of the noise cube
        std = convolved_noise_cube[i].std(axis=(1,2))/convolved_noise_cube[i].unit 
        mean_rms.append(np.mean(std))
        
    return convolved_cube, convolved_noise_cube, mean_rms

In [11]:
%%capture
# Capture all warnings

# Beam convolution parameters
circular_beam_axis = 20 # 20'' x 20'' circular beam

# Convolve the noise and signal cubes to a circular beam
circular_beam = Beam(major=circular_beam_axis*u.arcsec, minor=circular_beam_axis*u.arcsec, pa=0*u.deg)

# Convolution and rms for velocity cube
vel_circular, noise_circular, mean_rms_circular = beam_convolution_and_mean_rms(vel_subcube, noise_velsubcube, circular_beam, detections)

# Convolution and rms for frequency cube
freq_circular, noise_circular, mean_rms_circular_freq = beam_convolution_and_mean_rms(subcube, noise_subcube, circular_beam, detections)

# 6. Masking the extracted subcubes with the smoothed cubes

Masking is done so that when the profile fitting is done, the software can destinguish the baseline from the HI emission of the profile clearly.

In [12]:
def masked_cube(cube, smooth_cube, rms, sigma):
    mask_cube = cube.with_mask(smooth_cube > sigma*rms*smooth_cube.unit)
    
    return mask_cube

In [13]:
%%capture

masked_subcubes_freq = [] # List to store masked subcubes with frequency axis
masked_subcubes_vel = [] # List to store masked subcubes with velocity axis
masked_sum_freq = [] # List to store integrated flux from masked frequency cubes
masked_sum_vel = [] # List to store integrated flux from masked velocity cubes

for i in range(len(detections)):
    masked_freq = masked_cube(subcube[i], freq_circular[i], mean_rms_circular_freq[i], sigma = 3)
    masked_subcubes_freq.append(masked_freq)
    masked_sum_freq.append(np.nan_to_num(masked_freq.sum(axis=(1,2))))
    
    masked_vel = masked_cube(vel_subcube[i], vel_circular[i], mean_rms_circular[i], sigma = 3)
    masked_subcubes_vel.append(masked_vel)
    masked_sum_vel.append(np.nan_to_num(masked_vel.sum(axis=(1,2))))

# 7. Calculating the error in the flux densities

Now that we have masked the cubes, we must find the error in the flux density (Jy/beam). We need to calculate the rms in each channel of the unmasked subcubes (which is just the standard deviation), i.e. calculate the rms of each plane (or image slice) that has constant frequency/velocity but varying RA and Dec in the cube. The rms in each channel is the error in the flux density to first order. And for Gaussian statistics, the rms should be roughly equal to the standard deviation.

Also, since the frequencies are defined by the correlator, they do not have any uncertainties.

In [14]:
%%capture 
# The capture line above will prevent this cell from printing out too many warnings

error = [] # Error in Jy/beam

# Loop through each detection
for i in range(len(detections)):
    error.append(subcube[i].std(axis=(1,2)))

To double check how the standard deviation was calculated, we can calculate it manually:

In [ ]:
# %%capture

# mean = subcube[0].mean(axis=(1,2))

# stddev = []

# for i in range(len(subcube[0])):
#     num = pow(subcube[0][i] - mean[i], 2).sum(axis=(0,1))
#     sig = np.sqrt(num/subcube[0][i].size)
    
#     stddev.append(sig)

The standard deviation should equal the rms, so we can confirm that:

In [ ]:
# %%capture

# rms = []

# for i in range(len(subcube[0])):
#     numsum = pow(subcube[0][i], 2).sum(axis=(0,1))
#     sqrroot = np.sqrt(numsum/subcube[0][i].size)

#     rms.append(sqrroot.value)

# 8. Optional: plotting the profiles from each subcube

Since the spectra of the profiles are low resolution, the convention is to plot them as flux density as a function of frequency. High resolution spectra are usually what have the x-axis in velocity (km/s). Note that since we are limited by the channel resolution, it is best to plot the spectra with a step plot with the middle of the step indication the measurement and the width indication the channel resolution.

In [ ]:
# pl.figure(figsize=(50,120))

# for i in range(len(detections)):
#     pl.subplot(15,2,i+1)

#     pl.step((freqs[i].to(u.MHz)).value, masked_sum_freq[i].value, 'k', where='mid')
#     pl.errorbar((freqs[i].to(u.MHz)).value, masked_sum_freq[i].value, yerr=error[i].value, fmt='', marker=None, 
#                 ls='none', color='k')
#     pl.axvline(x=(center_freqs[i].to(u.MHz)).value, color='red', zorder=1, 
#                label='Center Frequency: %.2f MHz'%(center_freqs[i].to(u.MHz)).value)
#     pl.axhline(y=0, color='cyan', zorder=1, linestyle='--')
#     pl.tick_params(which='major', direction='in')
#     pl.title('Detection '+str(dets[i]))
#     pl.xlabel('Frequency (MHz)')
#     pl.ylabel('Integrated Flux (Jy/beam)')
#     pl.tight_layout()
#     pl.legend()
# pl.subplots_adjust(hspace=0.5)

# 9. Converting Jy/beam to Jy

To start doing the analysis on the spectra, we first need to convert the flux density in Jy/beam to Jy. The method behing this conversion is described in an excerpt from Meyer 2017:

![jybeamconv.png](jybeamconv.png)

$\Omega_B$ is the solid angle of the source and has the relation with the physical area given by

\begin{align}
\Omega_B = \frac{\pi b_\text{maj}b_{min}}{4\text{ln}2}, \\
\end{align}

in units of steradians (radians squared) or m$^2$m$^{-2}$. In our case, we will work in arcseconds squared. We can convert the errors in Jy/beam in the same way.

In [15]:
# A function that converts Jy/beam to Jy
def jybeam_jy(subcube_list, sum_list, err):
    '''
    subcube_list is the list of subcubes
    sum_list is the list of sum flux densities in Jy/beam
    err is the list of errors in Jy/beam
    '''
    # List to store the fluxes in Jy for each detection
    jy_list = []
    jy_err_list = []
    
    # Loop through each detection
    for i in range(len(subcube_list)):
        # List to store Jy for each channel
        jy = [] 
        jy_err = []
        
        # Loop through each channel
        for j in range(len(subcube_list[i])):
            # Calculate the solid angle or beam (omegaB) in arcsec squared
            # Jy/beam * beam = Jy/pixel_area
            bmaj = (subcube_list[i][j].beam.major).to(u.arcsec).value
            bmin = (subcube_list[i][j].beam.minor).to(u.arcsec).value

            omegaB = (np.pi*bmaj*bmin)/(4*np.log(2))

            # Calculate the pixel area in arcsec squared
            # Jy/pixel * pixel_area = Jy
            # CDELT1 and CDELT2 have the same magnitudes in degrees, so CDELT2 will be used since CDELT2 > 0
            pix_area = pow((subcube_list[i].header['CDELT2']*3600),2)

            jy.append(sum_list[i][j].value*(pix_area/omegaB))
            jy_err.append(err[i][j].value*(pix_area/omegaB))

        jy_list.append(np.array(jy)*u.Jy)
        jy_err_list.append(np.array(jy_err)*u.Jy)
    
    return jy_list, jy_err_list

In [16]:
%%capture
# Capture all warnings

flux, flux_err = jybeam_jy(masked_subcubes_freq, masked_sum_freq, error)

We can also calculate the flux of the unmasked profiles for later use.

In [17]:
%%capture
# Capture all warnings

unmasked_flux, unmasked_flux_err = jybeam_jy(subcube, Sum, error)

In [18]:
# Write the rms calculated from the unmasked profiles for each detection to a text file
dirName = analysis_res+'/users/wanga/mightee/analysis/COSMOS_1330/masked_profiles_COSMOS_1330_output/'

# Create the text file
f = open(dirName+'unmasked_rms.txt', 'w+')

f.write('Detection rms[Jy]\n')

# Loop through each detection
for i in range(len(detections)):
    # Only need the rms in Jy in a signal free part of the spectrum
    f.write('%i %e\n'%(detections[i][0], unmasked_flux_err[i][0].value))
        
f.close()

If you are working with the masked profiles, it is best to use the method above to also have errors for the values in the baseline that are zero as a result of masking. The method below gives the same errors, however, should only be used when using the unmasked profiles.

To calculate the error in the flux we can use:

\begin{align}
u(\text{flux}) = \sqrt{\left(\text{flux}\frac{u(Sum)}{Sum}\right)^2}
\end{align}

In [19]:
# Calculate the error in the flux
flux_err = []

# Loop through each detection
for i in range(len(detections)):
    # Calculate the error in each channel
    err = []
    for j in range(len(flux[i])):
        # Return 0 where uncertainties are 0
        if flux[i][j].value == 0:
            err.append(0)
        else:
            err.append(flux[i][j].value*(error[i][j].value/masked_sum_freq[i][j].value))
        
    flux_err.append(np.array(err)*u.Jy)

Optional: plot the spectra in terms of flux in Jy

In [ ]:
# pl.figure(figsize=(50,120))

# for i in range(len(detections)):
#     pl.subplot(15,2,i+1)

#     pl.step((freqs[i].to(u.MHz)).value, flux[i].value, 'k', where='mid') 
#     pl.errorbar((freqs[i].to(u.MHz)).value, flux[i].value, yerr=flux_err[i].value, fmt='', marker=None, ls='none', color='k')
#     pl.axvline(x=(center_freqs[i].to(u.MHz)).value, color='red', zorder=1, 
#                label='Center Frequency: %.2f MHz'%(center_freqs[i].to(u.MHz)).value)
#     pl.axhline(y=0, color='cyan', zorder=1, linestyle='--')
#     pl.tick_params(which='major', direction='in')
#     pl.title('Detection '+str(dets[i]))
#     pl.xlabel('Frequency (MHz)')
#     pl.ylabel('Flux Density (Jy)')
#     pl.tight_layout()
#     pl.legend() 
# pl.subplots_adjust(hspace=0.5)

# 10. Analysis
This section will be useful for if you want to calculate some of the HI quantities yourself. The results from the notebook are written to text files at the very end.

## 10.1. Create a mask where the HI emission is
Note that since one Early Science channel is 208985.82 Hz wide, we are plotting the spectral as a step function (i.e. for every 208985.82 Hz, we have one flux value). The bounds of the the emission need to lie at the centers of the step.

Note, the channel width can be checked using

In [20]:
dnu = freqs[0][1] - freqs[0][0]
dnu

<Quantity 209005.57643318 Hz>

To find out what the channel width is in terms of velocity, which will be useful later, one can either use the velocity list that was calculated in the subcube extraction using `with_spectral_unit`:

In [21]:
vel[0][4]-vel[0][5]

<Quantity 46.14986194 km / s>

Or by converting the frequencies directly into velocities, and then calculating the difference (which is essentially what `with_spectral_unit` does). However, the channel wdith varies slightly across each detection due to the relativistic conversion from frequencies to velocities, so we need to calculate the average channel width for each detection:

In [22]:
# List to store average channel widths in velocity space
dv_det = []

for i in range (len(detections)):
    dv_det.append(np.average(abs(np.diff(vel[i]))))

Back to creating the mask:

In [26]:
# Frequency limits for each of the detections
# The number of bounds below will depend on the size of your detections list
f_1a, f_1b = detections[0]['MinSpec']*1e3, detections[0]['MaxSpec']*1e3
f_4a, f_4b = detections[1]['MinSpec']*1e3, detections[1]['MaxSpec']*1e3
f_5a, f_5b = detections[2]['MinSpec']*1e3, detections[2]['MaxSpec']*1e3
f_7a, f_7b = detections[3]['MinSpec']*1e3, detections[3]['MaxSpec']*1e3
f_8a, f_8b = detections[4]['MinSpec']*1e3, detections[4]['MaxSpec']*1e3
f_10a, f_10b = detections[5]['MinSpec']*1e3, detections[5]['MaxSpec']*1e3
f_11a, f_11b = detections[6]['MinSpec']*1e3, detections[6]['MaxSpec']*1e3
f_12a, f_12b = detections[7]['MinSpec']*1e3, detections[7]['MaxSpec']*1e3
f_13a, f_13b = detections[8]['MinSpec']*1e3, detections[8]['MaxSpec']*1e3
f_14a, f_14b = detections[9]['MinSpec']*1e3, detections[9]['MaxSpec']*1e3
f_17a, f_17b = detections[10]['MinSpec']*1e3, detections[10]['MaxSpec']*1e3
f_18a, f_18b = detections[11]['MinSpec']*1e3, detections[11]['MaxSpec']*1e3
f_19a, f_19b = detections[12]['MinSpec']*1e3, detections[12]['MaxSpec']*1e3
f_20a, f_20b = detections[13]['MinSpec']*1e3, detections[13]['MaxSpec']*1e3
f_21a, f_21b = detections[14]['MinSpec']*1e3, detections[14]['MaxSpec']*1e3
f_22a, f_22b = detections[15]['MinSpec']*1e3, detections[15]['MaxSpec']*1e3
f_23a, f_23b = detections[16]['MinSpec']*1e3, detections[16]['MaxSpec']*1e3
f_24a, f_24b = detections[17]['MinSpec']*1e3, detections[17]['MaxSpec']*1e3
f_25a, f_25b = detections[18]['MinSpec']*1e3, detections[18]['MaxSpec']*1e3
f_30a, f_30b = detections[19]['MinSpec']*1e3, detections[19]['MaxSpec']*1e3
f_31a, f_31b = detections[20]['MinSpec']*1e3, detections[20]['MaxSpec']*1e3
f_33a, f_33b = detections[21]['MinSpec']*1e3, detections[21]['MaxSpec']*1e3
f_34a, f_34b = detections[22]['MinSpec']*1e3, detections[22]['MaxSpec']*1e3
f_37a, f_37b = detections[23]['MinSpec']*1e3, detections[23]['MaxSpec']*1e3
f_39a, f_39b = detections[24]['MinSpec']*1e3, detections[24]['MaxSpec']*1e3
f_40a, f_40b = detections[25]['MinSpec']*1e3, detections[25]['MaxSpec']*1e3
f_48a, f_48b = detections[26]['MinSpec']*1e3, detections[26]['MaxSpec']*1e3
f_49a, f_49b = detections[27]['MinSpec']*1e3, detections[27]['MaxSpec']*1e3
f_54a, f_54b = detections[28]['MinSpec']*1e3, detections[28]['MaxSpec']*1e3

# Zip and store these limits in a tuple
f_limit = [[f_1a, f_1b],[f_4a, f_4b],[f_5a, f_5b],[f_7a, f_7b],[f_8a, f_8b],[f_10a, f_10b],[f_11a, f_11b],[f_12a, f_12b],[f_13a, f_13b],
           [f_14a, f_14b],[f_17a, f_17b],[f_18a, f_18b],[f_19a, f_19b],[f_20a, f_20b],[f_21a, f_21b],[f_22a, f_22b],[f_23a, f_23b],
           [f_24a, f_24b],[f_25a, f_25b],[f_30a, f_30b],[f_31a, f_31b],[f_33a, f_33b],[f_34a, f_34b],[f_37a, f_37b],[f_39a, f_39b],
           [f_40a, f_40b],[f_48a, f_48b],[f_49a, f_49b],[f_54a, f_54b]]
f_a, f_b = zip(*f_limit)

# Look for close matches to our limits in the data set
# Store these matches in lists
findex_a = []
findex_b = []
for i in range(len(detections)):
    x = freqs[i].to(u.MHz).value
    # Rather than demand an exact match, should find the closest one
    findex_b.append(min(range(len(x)), key=lambda k: abs(x[k]-f_a[i])))
    findex_a.append(min(range(len(x)), key=lambda k: abs(x[k]-f_b[i])))

In [ ]:
pl.figure(figsize=(50,120))

for i in range(len(detections)):
    pl.subplot(15,2,i+1)

    pl.step(freqs[i].to(u.MHz).value, flux[i].value, 'k', where='mid')  
    pl.errorbar((freqs[i].to(u.MHz)).value, flux[i].value, yerr=flux_err[i].value, fmt='', marker=None, ls='none', color='k')
    pl.axvline(x=(center_freqs[i].to(u.MHz)).value, color='red', zorder=1, 
               label='Center Frequency: %.2f MHz'%(center_freqs[i].to(u.MHz)).value)
    pl.axhline(y=0, color='cyan', zorder=1, linestyle='--')
    pl.tick_params(which='major', direction='in')
    pl.title('Detection '+str(dets[i]))
    pl.xlabel('Frequency (MHz)')
    pl.ylabel('Integrated Flux (Jy/beam)')
    ax = pl.gca()
    ax.axvspan(freqs[i].to(u.MHz).value[findex_a[i]], freqs[i].to(u.MHz).value[findex_b[i]], alpha=0.15, color='grey')
    pl.tight_layout()
    pl.legend()
pl.subplots_adjust(hspace=0.5)

The following analyisis is based on the the equations in Meyer (2017) and notes found here: https://www.cv.nrao.edu/course/astr534/HILine.html 

## 10.2. Systemic velocities, Hubble distances and redshifts
We need to calculate the systemic velocity using the rest frame motions (i.e. the motions of objects through space; this is the peculiar velocity)

\begin{align}
V(z) = c\frac{\nu_0^2 - \nu^2}{\nu_0^2 + \nu^2}.
\end{align}

It is not advised to use $V = cz = c\frac{\nu_0 - \nu}{\nu}$ because the deviations from this linear law become increasingly larger. The uncertaintity can be calculated using

\begin{align}
u(R) = \sqrt{\sum_{i=1}^N \left(u(w_i)\frac{\partial f}{\partial w_i}\right)^2},
\end{align}

where in our case

\begin{align}
\frac{\partial f}{\partial w_i} &= \frac{\partial V}{\partial \nu} = -\frac{4c\nu_0^2\nu}{(\nu^2 + \nu_0^2)^2}.
\end{align}

Therefore

\begin{align}
u(V) = \sqrt{\left(-u(\nu)\frac{4c\nu_0^2\nu}{(\nu^2 + \nu_0^2)^2}\right)^2}.
\end{align}

In [28]:
# Defining a function used for converting an observed frequency (Ghz) into velocity (km/s) for 21 cm emission
def radial_vel(obs_freq):
    '''
    obs_freq is the observed frequency of the emitted line in Ghz
    nu_o is the rest/emitted frequency in GHz
    v_opt is returned in km/s
    '''
    nu_0 = (hdulist[0].header['RESTFRQ']*u.Hz).to(u.GHz)
    v_sys = (const.c.to(u.km/u.s))*((pow(nu_0,2) - pow(obs_freq, 2))/(pow(nu_0,2) + pow(obs_freq, 2)))
    return v_sys

# Calculate the systemic velocities
central_vel = [radial_vel(cfreq) for cfreq in center_freqs]
    
# Calculate the uncertainties using above formula
vel_unc = []
for i in range(len(detections)):
    un = np.sqrt(pow((-dnu.to(u.GHz)*4*(const.c.to(u.km/u.s))*pow((hdulist[0].header['RESTFRQ']*u.Hz).to(u.GHz),2)*center_freqs[i])/
                     (pow((pow(center_freqs[i],2) + pow((hdulist[0].header['RESTFRQ']*u.Hz).to(u.GHz),2)),2)),2))
    
    vel_unc.append(un)

# Display the velocities
for i in range(len(central_vel)):
    print("Detection %d: %.2f +\- %.2f km/s"%(int(i+1),central_vel[i].value,vel_unc[i].value))

Detection 1: 13941.34 +\- 46.11 km/s
Detection 2: 13756.94 +\- 46.09 km/s
Detection 3: 9635.00 +\- 45.51 km/s
Detection 4: 8544.73 +\- 45.35 km/s
Detection 5: 11460.42 +\- 45.77 km/s
Detection 6: 13756.94 +\- 46.09 km/s
Detection 7: 13066.35 +\- 45.99 km/s
Detection 8: 12974.39 +\- 45.98 km/s
Detection 9: 8408.70 +\- 45.33 km/s
Detection 10: 9453.03 +\- 45.48 km/s
Detection 11: 13342.41 +\- 46.03 km/s
Detection 12: 9407.56 +\- 45.47 km/s
Detection 13: 13342.41 +\- 46.03 km/s
Detection 14: 8816.95 +\- 45.39 km/s
Detection 15: 13526.58 +\- 46.06 km/s
Detection 16: 13526.58 +\- 46.06 km/s
Detection 17: 12974.39 +\- 45.98 km/s
Detection 18: 16719.72 +\- 46.50 km/s
Detection 19: 8590.08 +\- 45.36 km/s
Detection 20: 7865.19 +\- 45.25 km/s
Detection 21: 7910.45 +\- 45.26 km/s
Detection 22: 17417.94 +\- 46.60 km/s
Detection 23: 7367.80 +\- 45.18 km/s
Detection 24: 7412.98 +\- 45.19 km/s
Detection 25: 8091.54 +\- 45.29 km/s
Detection 26: 9635.00 +\- 45.51 km/s
Detection 27: 17091.93 +\- 46.55 k

One can calculate the redshifts to the galaxies using $z=\frac{\nu_0-\nu}{\nu}=\frac{\nu_0}{\nu}-1$, and approximate the uncertainties in $z$ using the derivative formula above:

\begin{align}
u(z) &= \sqrt{\left(u(z)\frac{\partial z}{\partial \nu}\right)^2} \\
 &= \sqrt{\left[u(\nu)\left(-\frac{\nu_0}{\nu^2}\right)\right]^2} \\
 &= \sqrt{\left[u(\nu)\left(-\frac{\nu_0}{\nu^2}\right)\right]^2} \\
 &= -\frac{u(\nu)\nu_0}{\nu^2} \\
\end{align}

In [29]:
# Extract the redshifts
z = [detections[i]['z'] for i in range(len(detections))]
    
# Calculate the uncertainties in z
#unc_z = [z[i]*np.sqrt(pow(dnu/(hdr[0].header['RESTFRQ']*u.Hz),2) + pow(dnu/center_freqs[i].to(u.Hz),2)) for i in range(len(detections))]
unc_z = [(dnu.to(u.GHz)*(hdulist[0].header['RESTFRQ']*u.Hz).to(u.GHz))/pow(v,2) for v in center_freqs]
    
# Display the redshifts
for i in range(len(z)):
    print("Detection %d: %.6f +\- %.6f"%(int(i+1),z[i],unc_z[i]))

Detection 1: 0.047637 +\- 0.000161
Detection 2: 0.046991 +\- 0.000161
Detection 3: 0.032672 +\- 0.000157
Detection 4: 0.028920 +\- 0.000156
Detection 5: 0.038987 +\- 0.000159
Detection 6: 0.046991 +\- 0.000161
Detection 7: 0.044577 +\- 0.000161
Detection 8: 0.044256 +\- 0.000160
Detection 9: 0.028453 +\- 0.000156
Detection 10: 0.032045 +\- 0.000157
Detection 11: 0.045541 +\- 0.000161
Detection 12: 0.031888 +\- 0.000157
Detection 13: 0.045541 +\- 0.000161
Detection 14: 0.029856 +\- 0.000156
Detection 15: 0.046185 +\- 0.000161
Detection 16: 0.046185 +\- 0.000161
Detection 17: 0.044256 +\- 0.000160
Detection 18: 0.057417 +\- 0.000165
Detection 19: 0.029076 +\- 0.000156
Detection 20: 0.026589 +\- 0.000155
Detection 21: 0.026744 +\- 0.000155
Detection 22: 0.059890 +\- 0.000165
Detection 23: 0.024886 +\- 0.000155
Detection 24: 0.025040 +\- 0.000155
Detection 25: 0.027365 +\- 0.000155
Detection 26: 0.032672 +\- 0.000157
Detection 27: 0.058735 +\- 0.000165
Detection 28: 0.032829 +\- 0.000157
D

The luminosity distance $D_L$ to a galaxy is given by

\begin{align}
D_L = (1+z)D_C,
\end{align}

where $D_C$ is the comoving distance given by

\begin{align}
D_C = \frac{c}{H_0}\int^z_0 \frac{dz'}{\sqrt{\Omega_m(1+z)^3 + \Omega_\Lambda}}
\end{align}

For MIGHTEE, the cosmological constants used to initialise ```cosmo``` are $H_0$, $\Omega_m$ and $\Omega_\Lambda$, and that is why $E(z)$ reduces to what it is.

In [30]:
# Cosmology parameters based on MIGHTEE
H0 = 67.4 # km/s/Mpc
unc_H0 = 0.54 #*u.km/u.s/u.Mpc # Uncertainty in the Hubble constant https://arxiv.org/pdf/1807.06209.pdf
h = H0/100
Om0 = 0.315
Ode0 = 0.685

cosmo = LambdaCDM(H0, Om0, Ode0)

# Calculate the luminosity distance
D = [(cosmo.comoving_distance(redshift))*(1 + redshift) for redshift in z]
    
# Calculate the uncertainty in D
unc_D = [D[i]*np.sqrt(pow(dv_det[i]/central_vel[i],2) + pow(unc_H0/H0,2)) for i in range(len(detections))]


# Display the Hubble distances
for i in range(len(D)):
    print("Detection %d: %.3f +\- %.3f Mpc"%(dets[i],D[i].value,unc_D[i].value))

Detection 1: 219.461 +\- 1.902 Mpc
Detection 4: 216.386 +\- 1.879 Mpc
Detection 5: 148.906 +\- 1.385 Mpc
Detection 7: 131.446 +\- 1.263 Mpc
Detection 8: 178.502 +\- 1.598 Mpc
Detection 10: 216.386 +\- 1.879 Mpc
Detection 11: 204.916 +\- 1.793 Mpc
Detection 12: 203.394 +\- 1.782 Mpc
Detection 13: 129.279 +\- 1.248 Mpc
Detection 14: 145.982 +\- 1.364 Mpc
Detection 17: 209.492 +\- 1.827 Mpc
Detection 18: 145.250 +\- 1.359 Mpc
Detection 19: 209.492 +\- 1.827 Mpc
Detection 20: 135.793 +\- 1.293 Mpc
Detection 21: 212.552 +\- 1.850 Mpc
Detection 22: 212.552 +\- 1.850 Mpc
Detection 23: 203.394 +\- 1.782 Mpc
Detection 24: 266.351 +\- 2.259 Mpc
Detection 25: 132.170 +\- 1.268 Mpc
Detection 30: 120.645 +\- 1.190 Mpc
Detection 31: 121.362 +\- 1.195 Mpc
Detection 33: 278.305 +\- 2.351 Mpc
Detection 34: 112.776 +\- 1.138 Mpc
Detection 37: 113.487 +\- 1.143 Mpc
Detection 39: 124.237 +\- 1.214 Mpc
Detection 40: 148.906 +\- 1.385 Mpc
Detection 48: 272.717 +\- 2.308 Mpc
Detection 49: 149.639 +\- 1.390 M

## 8.2. HI Mass
In order to calculate the HI mass $M_\text{HI}$, we need to use the formula

\begin{equation}
\left(\frac{M_\text{HI}}{\text{M}_\odot}\right) \approx \frac{2.36 \times 10^5}{(1+z)^2} \left(\frac{D_L}{\text{Mpc}}\right)^2 \int\left[\frac{S(\nu)}{\text{Jy}}\right]\left(\frac{dv}{\text{km/s}}\right) \\
\end{equation}

where $\int S(\nu)dv$ over the line is called the line flux and is in units of Jy km $\text{s}^{-1}$. However, we need only calculate the HI mass of the detection and not the whole spectrum. So let us define the limits of the profile, where the x-values are now in terms of velocity (km/s) and not frequency (MHz). However, since we already found the limits in terms of frequencies, we can flip `index_a` and `index_b` for the velocities.

In [31]:
index_a = findex_b
index_b = findex_a

In [32]:
def hi_mass_vel(DL, S, unmasked_S, z, vel_array, vel_lower, vel_upper, v_diff):
    '''
    DL is the list of luminosity distances in Mpc
    S is the list of masked fluxes in Jy
    unmasked_S is the list of unmasked fluxes in Jy
    vel_arr is the array of velocities for each subcube in km/s
    vel_lower is the list of lower limits for each profile
    vel_upper is the list of upper limits for each profile
    v_diff is the channel width in terms of velocity
    '''
    HI_mass = [] # List to store Mhi for each detection
    HI_mass_err = [] # List to store errors in Mhi
    line_flux = [] # List to store the line fluxes in km/s
    line_flux_err = [] # List to store errors in line fluxes
    
    # Loop through each data cube
    for i in range(len(detections)):
        # HI mass
        M = detections[i]['MHI(1e9)']*1e9
        HI_mass.append(M*u.M_sun)
        
        # Integrated flux
        Sdv = 0 # Initialise line flux
        
        # Loop through data in each cube to calculate the line flux
        for j in range(vel_lower[i], vel_upper[i]+1):
            # Velocity width
            dv = v_diff[i].value
            
            # Calculate the integral
            Sdv += S[i][j].value*dv
           
        # Sdv = (M*pow(1+z[i], 2))/(2.365*1e5*pow(D[i].value, 2))
        line_flux.append(Sdv*u.Jy*u.km/u.s) # Masked line flux
        
        # Calculate the error in the line flux from unmasked fluxes
        # Flux from off-emission channels 
        off_flux = np.array(unmasked_S[i].value[0:vel_lower[i]].tolist() + unmasked_S[i].value[vel_upper[i]:-1].tolist())
        
        # Calculate the std deviation of off-emission channels
        std_off_flux = np.std(off_flux)
        
        # Error in integrated flux
        line_err = (std_off_flux*np.sqrt(len(unmasked_S[i]))*v_diff[i].value)
        line_flux_err.append(line_err*u.Jy*u.km/u.s)
        
        # Calculate the HI mass error
        M_err = ((2.356*(10**5))/pow(1+z[i],2))*pow(DL[i].value,2)*line_err
        HI_mass_err.append(M_err*u.M_sun)
        
    return HI_mass, HI_mass_err, line_flux, line_flux_err

In [33]:
HI_mass1330_vel, HI_mass1330_vel_err, line_jykms, line_jykms_err = hi_mass_vel(D, flux, unmasked_flux, z, vel, index_a, index_b, dv_det)

[print('Detection %d: ('%(dets[i])+str(round(HI_mass1330_vel[i].value/1e9, 4))+' +/- '+str(round(HI_mass1330_vel_err[i].value/1e9, 4))
 +') x 10^9 Msol') for i in range(len(HI_mass1330_vel))]

Detection 1: (0.9616 +/- 0.6842) x 10^9 Msol
Detection 4: (5.1523 +/- 1.0717) x 10^9 Msol
Detection 5: (2.9648 +/- 0.3672) x 10^9 Msol
Detection 7: (0.6324 +/- 0.1467) x 10^9 Msol
Detection 8: (2.0893 +/- 0.7856) x 10^9 Msol
Detection 10: (1.1695 +/- 0.3027) x 10^9 Msol
Detection 11: (0.3097 +/- 0.2962) x 10^9 Msol
Detection 12: (1.8793 +/- 0.3064) x 10^9 Msol
Detection 13: (1.7701 +/- 0.1476) x 10^9 Msol
Detection 14: (4.8978 +/- 1.2177) x 10^9 Msol
Detection 17: (5.3088 +/- 1.165) x 10^9 Msol
Detection 18: (0.4295 +/- 0.2889) x 10^9 Msol
Detection 19: (2.8249 +/- 0.8586) x 10^9 Msol
Detection 20: (0.7943 +/- 1.5659) x 10^9 Msol
Detection 21: (6.1944 +/- 4.5715) x 10^9 Msol
Detection 22: (9.6605 +/- 2.8759) x 10^9 Msol
Detection 23: (5.8749 +/- 0.6854) x 10^9 Msol
Detection 24: (0.3199 +/- 1.2601) x 10^9 Msol
Detection 25: (1.3062 +/- 0.1441) x 10^9 Msol
Detection 30: (3.8548 +/- 1.6109) x 10^9 Msol
Detection 31: (0.7447 +/- 0.4355) x 10^9 Msol
Detection 33: (1.6711 +/- 1.2723) x 10^9

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [34]:
[print('Detection %d: ('%(dets[i])+str(round(math.log(HI_mass1330_vel[i].value, 10),4))+' +/- '
 +str(round(0.434*HI_mass1330_vel_err[i].value/HI_mass1330_vel[i].value, 4))+') log(Msol)') for i in range(len(HI_mass1330_vel))]

Detection 1: (8.983 +/- 0.3088) log(Msol)
Detection 4: (9.712 +/- 0.0903) log(Msol)
Detection 5: (9.472 +/- 0.0537) log(Msol)
Detection 7: (8.801 +/- 0.1006) log(Msol)
Detection 8: (9.32 +/- 0.1632) log(Msol)
Detection 10: (9.068 +/- 0.1123) log(Msol)
Detection 11: (8.491 +/- 0.415) log(Msol)
Detection 12: (9.274 +/- 0.0708) log(Msol)
Detection 13: (9.248 +/- 0.0362) log(Msol)
Detection 14: (9.69 +/- 0.1079) log(Msol)
Detection 17: (9.725 +/- 0.0952) log(Msol)
Detection 18: (8.633 +/- 0.2919) log(Msol)
Detection 19: (9.451 +/- 0.1319) log(Msol)
Detection 20: (8.9 +/- 0.8556) log(Msol)
Detection 21: (9.792 +/- 0.3203) log(Msol)
Detection 22: (9.985 +/- 0.1292) log(Msol)
Detection 23: (9.769 +/- 0.0506) log(Msol)
Detection 24: (8.505 +/- 1.7096) log(Msol)
Detection 25: (9.116 +/- 0.0479) log(Msol)
Detection 30: (9.586 +/- 0.1814) log(Msol)
Detection 31: (8.872 +/- 0.2538) log(Msol)
Detection 33: (9.223 +/- 0.3304) log(Msol)
Detection 34: (8.626 +/- 0.1147) log(Msol)
Detection 37: (8.813 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [35]:
[print('Detection %d: ('%(dets[i])+str(round(line_jykms[i].value ,4))+' +/- '+str(round(line_jykms_err[i].value, 4))+
 ') Jy km/s') for i in range(len(line_jykms))]

Detection 1: (0.1222 +/- 0.0662) Jy km/s
Detection 4: (0.4299 +/- 0.1065) Jy km/s
Detection 5: (0.5362 +/- 0.075) Jy km/s
Detection 7: (0.1588 +/- 0.0381) Jy km/s
Detection 8: (0.2244 +/- 0.113) Jy km/s
Detection 10: (0.1081 +/- 0.0301) Jy km/s
Detection 11: (0.0296 +/- 0.0327) Jy km/s
Detection 12: (0.1882 +/- 0.0343) Jy km/s
Detection 13: (0.3669 +/- 0.0396) Jy km/s
Detection 14: (1.1404 +/- 0.2583) Jy km/s
Detection 17: (0.4276 +/- 0.1232) Jy km/s
Detection 18: (0.0862 +/- 0.0619) Jy km/s
Detection 19: (0.1785 +/- 0.0908) Jy km/s
Detection 20: (0.1681 +/- 0.3823) Jy km/s
Detection 21: (0.4622 +/- 0.4701) Jy km/s
Detection 22: (0.7378 +/- 0.2957) Jy km/s
Detection 23: (0.4832 +/- 0.0767) Jy km/s
Detection 24: (0.0404 +/- 0.0843) Jy km/s
Detection 25: (0.1725 +/- 0.0371) Jy km/s
Detection 30: (0.6309 +/- 0.4951) Jy km/s
Detection 31: (0.1235 +/- 0.1323) Jy km/s
Detection 33: (0.0643 +/- 0.0783) Jy km/s
Detection 34: (0.1007 +/- 0.0392) Jy km/s
Detection 37: (0.1539 +/- 0.043) Jy km/s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

Writing the results to a text file.

In [38]:
# Write the masses and errors to a text file
# Create the text file
mass_txt = open(analysis_res+'masked_profile_HImasses.txt', 'w+')

# Loop through each detection
for i in range(len(detections)): 
    mass_txt.write('%d %f %f %f %f\n'%(dets[i], math.log(HI_mass1330_vel[i].value, 10), 0.434*HI_mass1330_vel_err[i].value/HI_mass1330_vel[i].value, central_vel[i].value, vel_unc[i].value))
        
mass_txt.close()

In [39]:
# Write the redshifts and errors to a text file
# Create the text file
z_txt = open(analysis_res+'COSMOS_1330_redshifts.txt', 'w+')

# Loop through each detection
for i in range(len(detections)):
    det = detections[i]['Detection'] # Detection number
    
    z_txt.write('%d %f %f %f %f\n'%(det,z[i],unc_z[i], D[i].value, unc_D[i].value))

z_txt.close()

In [40]:
index_a = findex_b
index_b = findex_a

# Loop through each detection
for i in range(len(detections)):
    # Create the text file
    f = open(analysis_res+'COSMOS_1330_masked_detection'+str(dets[i])+'_profile_vel.txt', 'w+')
    
    # Write the central velocity
    f.write('%s %f %f\n'%('CVel', central_vel[i].value, vel_unc[i].value))
    
    # Write indices strict to where emission is
    #f.write('%s %i %i\n'%('Indices', index_a[i], index_b[i]))
    
    # Write the integrated fluxes
    f.write('%s %f %f\n'%('Sdv', line_jykms[i].value, line_jykms_err[i].value))
    
    # Loop through each channel
    for j in range(len(masked_subcubes_vel[i])):
        f.write('%f %f %f\n'%(vel[i][j].value, flux[i][j].value, flux_err[i][j].value))
        
    f.close()